# Single Month Analysis

This notebook only processes a single month of data rather undertaking an analysis of a whole year. This will be more efficient when just running a single month (i.e., maintaining the alerts) but should not be used when processing multiple months. 

In [1]:
from dask_gateway import GatewayCluster
import dask.distributed
import planetary_computer
from pystac_client import Client
import odc.stac
import geopandas
import numpy
import rasterio
import xarray
import os
import json
import urllib
import time
from azure.storage.blob import BlobClient
from tqdm.notebook import tqdm

In [2]:
def apply_sen2_vld_msk(scns_xa, bands, qa_pxl_msk="SCL", out_no_data_val=0):
    scns_lcl_xa = scns_xa.copy()
    for band in bands:
        scns_lcl_xa[band].values[
            scns_lcl_xa[qa_pxl_msk].values == 0
        ] = out_no_data_val  # No Data
        scns_lcl_xa[band].values[
            scns_lcl_xa[qa_pxl_msk].values == 1
        ] = out_no_data_val  # Saturation
        scns_lcl_xa[band].values[
            scns_lcl_xa[qa_pxl_msk].values == 2
        ] = out_no_data_val  # Cast Shadow
        scns_lcl_xa[band].values[
            scns_lcl_xa[qa_pxl_msk].values == 3
        ] = out_no_data_val  # Cloud Shadows
        scns_lcl_xa[band].values[
            scns_lcl_xa[qa_pxl_msk].values == 8
        ] = out_no_data_val  # Cloud Medium Probability
        scns_lcl_xa[band].values[
            scns_lcl_xa[qa_pxl_msk].values == 9
        ] = out_no_data_val  # Cloud High Probability
        scns_lcl_xa[band].values[
            scns_lcl_xa[qa_pxl_msk].values == 10
        ] = out_no_data_val  # Thin Cirrus
    return scns_lcl_xa


def apply_sen2_offset(sen2_scns_xa, offset=-1000):

    # Define the date splitting whether the offset should be applied.
    off_date = numpy.datetime64("2022-01-25")
    # Get Minimum date in timeseries
    time_min = sen2_scns_xa.time.min().values
    # Get Maximum date in timeseries
    time_max = sen2_scns_xa.time.max().values

    # Get the list of variables
    bands = list(sen2_scns_xa.data_vars)
    # List of all bands for which offset should be applied if present.
    s2_img_bands = [
        "B01",
        "B02",
        "B03",
        "B04",
        "B05",
        "B06",
        "B07",
        "B08",
        "B8A",
        "B09",
        "B10",
        "B11",
        "B12",
    ]

    if (time_min < off_date) and (time_max > off_date):
        # Crosses the offset data and therefore part of the dataset needs offset applying
        sen2_scns_xa_pre_off = sen2_scns_xa.sel(time=slice(time_min, off_date))
        sen2_scns_xa_post_off = sen2_scns_xa.sel(time=slice(off_date, time_max))
        for band in bands:
            if band in s2_img_bands:
                sen2_scns_xa_post_off[band] = sen2_scns_xa_post_off[band] + offset
                sen2_scns_xa_post_off[band].where(sen2_scns_xa_post_off[band] < 0, 0)
                sen2_scns_xa_post_off[band].where(
                    sen2_scns_xa_post_off[band] > 10000, 0
                )
        sen2_scns_xa = xarray.concat(
            [sen2_scns_xa_pre_off, sen2_scns_xa_post_off], dim="time"
        )
    elif time_min > off_date:
        # All scenes after offset date apply to all
        for band in bands:
            if band in s2_img_bands:
                sen2_scns_xa[band] = sen2_scns_xa[band] + offset
                sen2_scns_xa[band].where(sen2_scns_xa[band] < 0, 0)
                sen2_scns_xa[band].where(sen2_scns_xa[band] > 10000, 0)
    # else: time_max < off_date:
    # Do nothing - no offset required
    return sen2_scns_xa


def get_img_metadata(img_file):
    img_data_obj = rasterio.open(img_file)
    img_bounds = img_data_obj.bounds
    img_bbox = [img_bounds.left, img_bounds.bottom, img_bounds.right, img_bounds.top]
    img_x_res, img_y_res  = img_data_obj.res
    if img_y_res > 0:
        img_y_res = img_y_res * (-1)
    img_data_obj = None
    return img_bbox, img_x_res, img_y_res


def get_img_band_array(img_file, band=1):
    img_data_obj = rasterio.open(img_file)
    img_arr = img_data_obj.read(band)
    img_data_obj = None
    return img_arr


def find_month_end_date(year, month):
    import calendar
    cal = calendar.Calendar()
    month_days = cal.monthdayscalendar(year, month)
    max_day_month = numpy.array(month_days).flatten().max()
    return max_day_month


def zero_pad_num_str(
    num_val: float,
    str_len: int = 3,
    round_num: bool = False,
    round_n_digts: int = 0,
    integerise: bool = False,
    absolute: bool = False,
    gain: float = 1,
) -> str:
    if absolute:
        num_val = abs(num_val)
    if round_num:
        num_val = round(num_val, round_n_digts)
    if integerise:
        num_val = int(num_val * gain)

    num_str = "{}".format(num_val)
    num_str = num_str.zfill(str_len)
    return num_str


def test_asset_urls(signed_items):
    chkd_items = list()
    for scn_item in tqdm(signed_items):
        assets_present = True
        for asset_name in scn_item.assets:
            try:
                if (
                    urllib.request.urlopen(scn_item.assets[asset_name].href).getcode()
                    != 200
                ):
                    assets_present = False
                    break
            except urllib.error.HTTPError:
                assets_present = False
                break
            time.sleep(0.1)
        if assets_present:
            chkd_items.append(scn_item)
    print(f"Before: {len(signed_items)}")
    print(f"After: {len(chkd_items)}")
    return chkd_items


In [3]:
def find_recent_alerts(tile, gmw_ext_msk, start_year, end_year, c_year, c_month, out_dir, sas_token_info):
    years = numpy.arange(start_year, end_year, 1)
    found_tile_alerts = False
    found_miss_alerts = False
    alerts_year = 0
    alerts_month = 0

    for year in years:
        for month_idx in numpy.arange(0, 12, 1):
            month = month_idx + 1
            month_end_day = find_month_end_date(year, month)
            month_str = zero_pad_num_str(month, str_len = 2, round_num = False, round_n_digts = 0)
            month_end_day_str = zero_pad_num_str(month_end_day, str_len = 2, round_num = False, round_n_digts = 0)
            date_str = f"{year}{month_str}"
            if (year == c_year) and (month == c_month):
                # Cannot process beyond current month of current year as there will be no data - its the future...
                break

            out_alerts_img_file = f"gmw_{tile}_{date_str}_chg_alerts.tif"
            out_alerts_img_file_url = os.path.join(sas_token_info["url"], out_dir, out_alerts_img_file)
            out_alerts_img_file_url_signed = f"{out_alerts_img_file_url}?{sas_token_info['sas_token']}"
            out_alerts_img_exists =  BlobClient.from_blob_url(out_alerts_img_file_url_signed).exists()
            
            out_meta_img_file = f"gmw_{tile}_{date_str}_chg_alerts_meta.tif"
            out_meta_img_file_url = os.path.join(sas_token_info["url"], out_dir, out_meta_img_file)
            out_meta_img_file_url_signed = f"{out_meta_img_file_url}?{sas_token_info['sas_token']}"
            out_meta_img_exists =  BlobClient.from_blob_url(out_meta_img_file_url_signed).exists()
            
            if out_meta_img_exists and out_alerts_img_exists:
                found_tile_alerts = True
                meta_img_file = out_meta_img_file
                alerts_img_file = out_alerts_img_file
                alerts_year = year
                alerts_month_idx = month_idx
            else:
                found_miss_alerts = True
                break
        if found_miss_alerts:
            break
    
    if found_tile_alerts:
        start_month_idx = alerts_month_idx + 1
        if start_month_idx == 12:
            start_month_idx = 0
            alerts_year = alerts_year + 1
        years = numpy.arange(alerts_year, end_year, 1)
        # Define the signed URL
        meta_img_file_url = os.path.join(sas_token_info["url"], out_dir, meta_img_file)
        meta_img_file_url_signed = f"{meta_img_file_url}?{sas_token_info['sas_token']}"
        # Read the layers in.
        score_arr = get_img_band_array(meta_img_file_url_signed, band=1)
        date_year_arr = get_img_band_array(meta_img_file_url_signed, band=2)
        date_month_arr = get_img_band_array(meta_img_file_url_signed, band=3)
    else:
        score_arr = numpy.zeros_like(gmw_ext_msk, dtype=numpy.int16)
        date_year_arr = numpy.zeros_like(gmw_ext_msk, dtype=numpy.uint16)
        date_month_arr = numpy.zeros_like(gmw_ext_msk, dtype=numpy.uint8)
        start_month_idx = 0
    
    return years, start_month_idx, score_arr, date_year_arr, date_month_arr

In [4]:
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.
cluster.adapt(minimum=4, maximum=24)
print(cluster.dashboard_link)

client = dask.distributed.Client(cluster, timeout=5)
odc.stac.configure_rio(cloud_defaults=True, client=client)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.bb565f0a30c1426ab019344cc0743ef1/status


In [5]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

In [6]:
bands = ["B04", "B08", "SCL"]

In [7]:
# Define the tiles to be processed.
tiles_gdf = geopandas.read_file("../00_base_data/alert_region_tiles.geojson")
tiles = tiles_gdf["tile"].values
# use to indicate range of tiles for batch processing (e.g. 0:100 for the 1st 100 or 400: for tile 400 to the end)
#tiles = tiles[401:]

tiles = tiles.tolist()
#tiles.remove("N29W113")
#tiles.remove("N29W114")
#tiles.remove("N28W112")
#tiles.remove("N16W093")

# Change to just run some test tiles...
#tiles = ["N02W079"]

n_tiles = len(tiles)

In [8]:
# significantly slows processing, only use when running tiles that produce e.g. http 404 errors, change to True for running separate tiles
check_s2_data = False

In [9]:
sas_info_file = "/home/jovyan/azure_info.json"
with open(sas_info_file) as f:
    sas_token_info = json.load(f)

In [10]:
tmp_path = "monthly_change_imgs_tmp"
if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)

In [11]:
out_img_dir = "monthly_change_imgs"
gmw_base_dir = "gmw_2018_revised"

## Update the month to be processed here:

In [12]:
# The year and month to be processed - meta-data needs to be available for 
# the previous month (will be checked) and only the month of interest will 
# be processed.
c_year = 2022
c_month = 11

In [14]:
n_tile = 0
for tile in tiles:
    print(f"{tile}: ({n_tile+1} of {n_tiles})")
    
    # Path of the revised GMW 2018 extent image.
    gmw_tile_img_file = f"gmw_{tile}_2018_alerts_ext.tif"
    gmw_tile_img_url = os.path.join(sas_token_info["url"], gmw_base_dir, gmw_tile_img_file)
    gmw_tile_img_url_signed = f"{gmw_tile_img_url}?{sas_token_info['sas_token']}"
        
    # If the GMW 2018 tile does not exist then skip tile and go to the next one.
    if not BlobClient.from_blob_url(gmw_tile_img_url_signed).exists():
        print("\tNo GMW Extent Tile Available...")
        n_tile += 1
        continue
        
    # Output file names for the change alerts and associated metadata (i.e., score and date of first change observation).
    out_meta_img_file = f"gmw_{tile}_{date_str}_chg_alerts_meta.tif"
    out_alerts_img_file = f"gmw_{tile}_{date_str}_chg_alerts.tif"
    
    out_meta_img_file_url = os.path.join(sas_token_info["url"], out_img_dir, out_meta_img_file)
    out_meta_img_file_url_signed = f"{out_meta_img_file_url}?{sas_token_info['sas_token']}"
    out_meta_img_exists =  BlobClient.from_blob_url(out_meta_img_file_url_signed).exists()
    
    out_alerts_img_file_url = os.path.join(sas_token_info["url"], out_img_dir, out_alerts_img_file)
    out_alerts_img_file_url_signed = f"{out_alerts_img_file_url}?{sas_token_info['sas_token']}"
    out_alerts_img_exists =  BlobClient.from_blob_url(out_alerts_img_file_url_signed).exists()
    
    if out_meta_img_exists and out_alerts_img_exists:
        print("\tOutput images already exist...")
        n_tile += 1
        continue
    
    # Get image tile meta-data
    bbox, img_x_res, img_y_res = get_img_metadata(gmw_tile_img_url_signed)
    # Get GMW Mask as array
    gmw_ext_msk = get_img_band_array(gmw_tile_img_url_signed, band=2)
    
    # Check there are mangrove pixels within the scene (mangroves == 1)
    if numpy.max(gmw_ext_msk) == 0:
        print("\tNo Mangrove Pixels in Mask...")
        del gmw_ext_msk
        n_tile += 1
        continue

    # Get the image shape (i.e., number of pixels)
    img_shp = gmw_ext_msk.shape

    # Define the output image spatial transformation.
    out_img_transform = rasterio.transform.Affine(img_x_res, 0.0, bbox[0], 0.0, img_y_res, bbox[3])
    
    # Find when the latest alerts have been created and continue from any previous alerts
    # This is required for if the analysis crashes or when rerun each month for new alerts
    years, start_month_idx, score_arr, date_year_arr, date_month_arr = find_recent_alerts(tile, gmw_ext_msk, c_year, c_year+1, c_year, c_month, out_img_dir, sas_token_info)
    print(f"\tYears: {years}; Start Month: {start_month_idx}")
    
    # Create an empty array for the binary mask for the monthly alerts
    month_alerts_arr = numpy.zeros_like(score_arr, dtype=numpy.uint8)
    
    month_str = zero_pad_num_str(c_month, str_len=2, round_num = False, round_n_digts = 0)
    month_end_day = find_month_end_date(c_year, c_month)
    month_end_day_str = zero_pad_num_str(month_end_day, str_len=2, round_num = False, round_n_digts = 0)
    month_idx = c_month - 1
    date_str = f"{c_year}{month_str}"
    
    ####################################################################
    # Date range of the ROI (Beginning of Jan to end of Dec)
    chng_time_range = f"{c_year}-{month_str}-01/{c_year}-{month_str}-{month_end_day_str}"
    # Find the scenes for the year of interest.
    chng_search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=chng_time_range, query={"eo:cloud_cover": {"lt": 50}})
    chng_items = chng_search.get_all_items()
    n_chng_items = len(chng_items)
    print(f"\t{chng_time_range}: N Items = {n_chng_items}")
    ####################################################################

    ####################################################################
    # Read the scenes into dask array structure and make persistant in memory
    signed_chng_items = [planetary_computer.sign(item) for item in chng_items]
    if check_s2_data:
        signed_chng_items = test_asset_urls(signed_chng_items)
    
    if len(signed_chng_items) > 0:
        sen2_chng_scn_xa = odc.stac.stac_load(
            signed_chng_items,
            bands=bands,
            groupby="solar_day",
            chunks={"time":12, "latitude": 1024, "longitude": 1024},
            bbox=bbox,
            crs="EPSG:4326",
            resolution=img_x_res
        )

        # Comment out for larger datasets which don't fit into memory.
        sen2_chng_scn_xa = sen2_chng_scn_xa.persist()

        # Apply Offset
        sen2_chng_scn_xa = apply_sen2_offset(sen2_chng_scn_xa)

        # Apply cloud mask to all scenes
        sen2_chng_scn_xa = sen2_chng_scn_xa.map_blocks(apply_sen2_vld_msk, kwargs={"bands":bands})
        ####################################################################

        ####################################################################
        # 'Clean' up the Red and NIR bands to remove any values less than zero."
        sen2_chng_scn_xa['B04'] = sen2_chng_scn_xa.B04.where(sen2_chng_scn_xa.B04>0)
        sen2_chng_scn_xa['B08'] = sen2_chng_scn_xa.B08.where(sen2_chng_scn_xa.B08>0)

        # Valid pixel mask
        sen2_chng_vld_xa = sen2_chng_scn_xa.B04>0

        # Calculate the NDVI
        ndvi_chng_scn_xa = ((sen2_chng_scn_xa.B08-sen2_chng_scn_xa.B04)/(sen2_chng_scn_xa.B08+sen2_chng_scn_xa.B04))
        ####################################################################


        ####################################################################
        # Date range of the year before ROI for checking changes have changed over the last 12 months.
        ref_time_range = f"{c_year-1}-{month_str}-01/{c_year-1}-{month_str}-{month_end_day_str}"
        # Search for reference scenes from year earlier than ROI.
        ref_search = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox, datetime=ref_time_range, query={"eo:cloud_cover": {"lt": 50}})
        ref_items = ref_search.get_all_items()
        n_ref_items = len(ref_items)
        print(f"\t{ref_time_range}: N Items = {n_ref_items}")
        ####################################################################

        ####################################################################
        # Read the scenes into dask array structure and make persistant in memory
        signed_ref_items = [planetary_computer.sign(item) for item in ref_items]
        if check_s2_data:
            signed_ref_items = test_asset_urls(signed_ref_items)
        
        if len(signed_ref_items) > 0:
            sen2_ref_scn_xa = odc.stac.stac_load(
                signed_ref_items,
                bands=bands,
                groupby="solar_day",
                chunks={"time":12, "latitude": 1024, "longitude": 1024},
                bbox=bbox,
                crs="EPSG:4326",
                resolution=img_x_res
            )

            # Comment out for larger datasets which don't fit into memory.
            sen2_ref_scn_xa = sen2_ref_scn_xa.persist()

            # Apply Offset
            sen2_ref_scn_xa = apply_sen2_offset(sen2_ref_scn_xa)

            # Apply cloud mask to all scenes
            sen2_ref_scn_xa = sen2_ref_scn_xa.map_blocks(apply_sen2_vld_msk, kwargs={"bands":bands})
            ####################################################################

            ####################################################################
            # 'Clean' up the Red and NIR bands to remove any values less than zero."
            sen2_ref_scn_xa['B04'] = sen2_ref_scn_xa.B04.where(sen2_ref_scn_xa.B04>0)
            sen2_ref_scn_xa['B08'] = sen2_ref_scn_xa.B08.where(sen2_ref_scn_xa.B08>0)

            # Calculate the NDVI
            ndvi_ref_scn_xa = ((sen2_ref_scn_xa.B08-sen2_ref_scn_xa.B04)/(sen2_ref_scn_xa.B08+sen2_ref_scn_xa.B04))

            # Create monthly summaries
            monthly_ref_ndvi_xa = ndvi_ref_scn_xa.median(dim="time", skipna=True).compute()
            ####################################################################

            # Find the pixels of change for the month on a per-scene basis (i.e., NDVI < 0.25 and then difference in NDVI from a year earlier is > 0.15)
            monthly_chng_pxls_xa = numpy.logical_and((ndvi_chng_scn_xa < 0.25), numpy.abs(ndvi_chng_scn_xa - monthly_ref_ndvi_xa) > 0.15)

            # Get the valid masks for the current month
            monthly_vld_pxls_scn_xa = sen2_chng_vld_xa

            # Mask the GMW changes and valid masks to the GMW mask.
            gmw_monthly_chng_pxls_xa = monthly_chng_pxls_xa.where(gmw_ext_msk == 1)
            gmw_monthly_vld_pxls_scn_xa = monthly_vld_pxls_scn_xa.where(gmw_ext_msk == 1)

            # Sum the number of changes observed in the scenes
            gmw_monthly_chng_pxls_count_xa = gmw_monthly_chng_pxls_xa.sum(dim="time", skipna=True)
            # Sum the number of observations.
            gmw_monthly_vld_pxls_count_xa = gmw_monthly_vld_pxls_scn_xa.sum(dim="time", skipna=True)

            # Run the compute in dask - before this no analysis actually happens it is just building the analysis graph
            dask.compute(gmw_monthly_chng_pxls_count_xa, gmw_monthly_vld_pxls_count_xa)

            ####################################################################
            # Update the score data values (Add 1 for a change and -1 for no change).
            lcl_score_arr = score_arr + (gmw_monthly_chng_pxls_count_xa.values) - (gmw_monthly_vld_pxls_count_xa.values - gmw_monthly_chng_pxls_count_xa.values)
            # Any pixels with a score of 5 before the analysis set back to 5 - they have already been identified as a change.
            lcl_score_arr[score_arr == 5] = 5

            # Update the date layers so that any pixels which have gone back to a score of 0 are reset.
            date_year_arr[lcl_score_arr < 1] = 0
            date_month_arr[lcl_score_arr < 1] = 0

            # For pixels with a score > 0 without a date define, define the date as this is the first observation.
            date_year_arr[(lcl_score_arr > 0) & (date_year_arr == 0)] = c_year
            date_month_arr[(lcl_score_arr > 0) & (date_month_arr == 0)] = c_month

            # Identify new confirmed alerts (i.e., the score has now reach 5).
            month_alerts_arr[numpy.logical_and(score_arr<5, lcl_score_arr>4)] = 1

            # Cap the score layer at 5 and don't allow scores lower than 0.
            lcl_score_arr[lcl_score_arr>5] = 5
            lcl_score_arr[lcl_score_arr<1] = 0

            # Copy the local score image to replace the score image
            score_arr = numpy.copy(lcl_score_arr)
            ####################################################################

            # Delete Monthly Objects.
            del monthly_chng_pxls_xa
            del monthly_vld_pxls_scn_xa
            del gmw_monthly_chng_pxls_xa
            del gmw_monthly_vld_pxls_scn_xa
            del gmw_monthly_chng_pxls_count_xa
            del gmw_monthly_vld_pxls_count_xa
            del lcl_score_arr
            
            # Delete annual objects
            del sen2_ref_scn_xa
            del ndvi_ref_scn_xa
            del monthly_ref_ndvi_xa
            
        del sen2_chng_scn_xa
        del sen2_chng_vld_xa
        del ndvi_chng_scn_xa
            
    ####################################################################
    out_meta_img_file_tmp = os.path.join(tmp_path, out_meta_img_file)
    out_alerts_img_file_tmp = os.path.join(tmp_path, out_alerts_img_file)

    # Create the output image file for the metadata
    with rasterio.open(out_meta_img_file_tmp,
                                    'w',
                                    driver='COG',
                                    height=img_shp[0],
                                    width=img_shp[1],
                                    count=3,
                                    dtype=numpy.uint16,
                                    crs='epsg:4326',
                                    transform=out_img_transform,
                                ) as out_img_dataset:

        # Write output array to the image file
        out_img_dataset.write(score_arr, 1)
        # Name the image band.
        out_img_dataset.set_band_description(1, "Alerts_Score")

        # Write output array to the image file
        out_img_dataset.write(date_year_arr, 2)
        # Name the image band.
        out_img_dataset.set_band_description(2, "Alert_Date_Year")

        # Write output array to the image file
        out_img_dataset.write(date_month_arr, 3)
        # Name the image band.
        out_img_dataset.set_band_description(3, "Alert_Date_Month")
    ####################################################################


    ####################################################################
    # Create the output image file for the newly identified alerts.
    with rasterio.open(out_alerts_img_file_tmp,
                                    'w',
                                    driver='COG',
                                    height=img_shp[0],
                                    width=img_shp[1],
                                    count=1,
                                    dtype=numpy.uint8,
                                    crs='epsg:4326',
                                    transform=out_img_transform,
                                ) as out_img_dataset:

        # Write output array to the image file
        out_img_dataset.write(month_alerts_arr, 1)
        # Name the image band.
        out_img_dataset.set_band_description(1, "Alerts")
    ####################################################################

    if os.path.exists(out_meta_img_file_tmp):
        blob_client = BlobClient.from_blob_url(out_meta_img_file_url_signed)
        with open(out_meta_img_file_tmp, 'rb') as data:
            blob_client.upload_blob(data)
        blob_client = None
        rasterio.shutil.delete(out_meta_img_file_tmp, driver="COG")

    if os.path.exists(out_alerts_img_file_tmp):
        blob_client = BlobClient.from_blob_url(out_alerts_img_file_url_signed)
        with open(out_alerts_img_file_tmp, 'rb') as data:
            blob_client.upload_blob(data)
        blob_client = None
        rasterio.shutil.delete(out_alerts_img_file_tmp, driver="COG")

    print(f"\tCompleted: {c_year} - {month_str}")
    
    # Delete tile based arrays
    del month_alerts_arr
    del score_arr
    del date_year_arr
    del date_month_arr
    del gmw_ext_msk
    # Increment the tile number for user feedback.
    n_tile += 1
    # Restart the dask workers to ensure all the memory etc. is cleared.
    if n_tile % 10 == 0:
        client.restart(wait_for_workers=False)

S05W081: (1 of 484)
	Output images already exist...
N01W081: (2 of 484)
	Output images already exist...
N02W080: (3 of 484)
	Output images already exist...
S03W081: (4 of 484)
	Output images already exist...
N02W079: (5 of 484)
	Output images already exist...
S03W080: (6 of 484)
	Output images already exist...
N01W080: (7 of 484)
	Output images already exist...
S02W080: (8 of 484)
	Output images already exist...
S04W082: (9 of 484)
	Output images already exist...
N00W081: (10 of 484)
	Output images already exist...
S02W081: (11 of 484)
	Output images already exist...
N22W091: (12 of 484)
	Output images already exist...
N19W104: (13 of 484)
	Output images already exist...
N19W089: (14 of 484)
	Output images already exist...
N17W095: (15 of 484)
	Output images already exist...
N19W094: (16 of 484)
	Output images already exist...
N21W098: (17 of 484)
	Output images already exist...
N21W097: (18 of 484)
	Output images already exist...
N19W103: (19 of 484)
	Output images already exist...
N2

In [ ]:
# Close the dask cluster
client.close()
cluster.close()

In [ ]:
#if os.path.exists(tmp_path):
#    shutil.rmtree(tmp_path)